In [1]:
import openai
from tqdm import tqdm
import os
import pandas as pd
import snowflake.connector

api_key_file = '/Users/vishalkumar/Documents/apikey.txt'
if os.path.isfile(api_key_file):
    with open(api_key_file) as f:
        openai.api_key = f.readline()
else:
    print(f"Error: {api_key_file} not found.")

OPENAI_API_KEY = openai.api_key

#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [6]:
#Get data from snowflake
sql = f'''
with RankedTasks AS (
select
ta.task,
u.email,
t.params:templateVariables.image_url::string as Image_Url,
case when ta.response:responses[3]:context:response:annotations:prompt:response[0]::string is null 
then ta.response:responses[2]:output::string
else ta.response:responses[3]:context:response:annotations:prompt:response[0]::string 
end as PROMPT,
case when ta.response:responses[3]:context:response:annotations:response:response[0]::string is null
then ta.response:responses[3]:output::string
else ta.response:responses[3]:context:response:annotations:response:response[0]::string
end as response_generated
,ROW_NUMBER() OVER (PARTITION BY ta.TASK ORDER BY ta.ATTEMPTED_AT DESC) AS rn
from scale_prod.public.taskattempts ta 
JOIN scale_prod.public.TASKS t ON ta.TASK = t._id
JOIN scale_prod.public.projects p ON p._id = ta.project
JOIN scale_prod.public.PIPELINEV3HUMANNODES pv3 ON pv3.task = t._id
JOIN scale_prod.public.users u on u._id = ta.attempted_by
where p._ID = '65e23dd219c580a16e44e374'
and ta.task IN ('65e9196eac3aa23880139ede',
'65e9113cecc09a673e691ec4',
'65e9195aba9c50de8bd1fa37'))
select
task,
email,
image_url,
PROMPT,
response_generated
FROM RankedTasks
WHERE rn = 1
'''
cs.execute(sql)
idf = cs.fetch_pandas_all()

In [7]:
def vision_gpt(image_url, prompt):
    import openai
    client = openai
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url,
                            "detail": "high"
                        },
                    },
                ],
            }
        ],
        max_tokens=1024,
    )
    return response

In [8]:
n = idf.shape[0]
print(f"Number of rows to process: {n}")

Number of rows to process: 375


In [9]:
idf['SOTA_RESPONSE'] = None
for i, row in tqdm(idf.iterrows(), total=idf.shape[0]):
    try:
        #if pd.isna(row['TRANSFORMED_IMAGE_URL']):
        image_url = row['IMAGE_URL']
        #else:
        #    image_url = row['TRANSFORMED_IMAGE_URL']
        prompt = row['PROMPT']
        response = vision_gpt(image_url, prompt)
        response = response.choices[0].message.content
        idf.loc[i, 'SOTA_RESPONSE'] = response
    except Exception as e:
        print(f"Error occurred: {e}")
        continue

#save idf to a csv file
idf.to_csv('Nightingale_MM_SFT_DI_SOTA16.csv', index=False)

 43%|████▎     | 160/375 [37:33<32:57,  9.20s/it]  

Error occurred: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image is below 20 MB in size and is of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'sanitizer_server_error'}}


 55%|█████▌    | 207/375 [47:23<21:28,  7.67s/it]  

Error occurred: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image is below 20 MB in size and is of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'sanitizer_server_error'}}


 55%|█████▌    | 208/375 [47:27<18:11,  6.54s/it]

...

 94%|█████████▍| 353/375 [1:24:35<02:39,  7.26s/it]

Error occurred: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image is below 20 MB in size and is of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'sanitizer_server_error'}}


 95%|█████████▍| 356/375 [1:25:02<02:26,  7.69s/it]

Error occurred: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image is below 20 MB in size and is of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'sanitizer_server_error'}}


100%|██████████| 375/375 [1:30:15<00:00, 14.44s/it]


In [121]:
idf.to_csv('Nightingale_MM_SFT_DI_SOTA22.csv')